In [1]:
%load_ext autoreload
%autoreload 2

# 0. Full text dataset

In [2]:
import pandas as pd

df_pmc_patients = pd.read_parquet('/s3/misha/data_dir/PMC_patients/full_texts_PMC-Patients-V2.parquet', engine='pyarrow')

In [3]:
df_pmc_patients

,PMID,patient_uid,title,full_text
0,15268761,497050-1,Echocardiographic assessment and percutaneous ...,Echocardiographic assessment and percutaneous ...
1,15268761,497050-2,Echocardiographic assessment and percutaneous ...,Echocardiographic assessment and percutaneous ...
2,15268761,497050-3,Echocardiographic assessment and percutaneous ...,Echocardiographic assessment and percutaneous ...
3,15272940,503399-1,A unique dedifferentiated tumor of the retrope...,A unique dedifferentiated tumor of the retrope...
4,15285782,509249-1,Adenoid cystic carcinoma of the parotid metast...,Adenoid cystic carcinoma of the parotid metast...
...,...,...,...,...
250289,38881976,11179526-1,Subarachnoid hemorrhage mimicking an acute mig...,Subarachnoid hemorrhage mimicking an acute mig...
250290,38881977,11179541-1,Deficiency of adenosine deaminase 2 leading to...,Deficiency of adenosine deaminase 2 leading to...
250291,38883244,11179659-1,Successful Undergoing Esophagogastric Anastomo...,Successful Undergoing Esophagogastric Anastomo...
250292,38881774,11180348-1,A case of acute hypercapnic respiratory failur...,A case of acute hypercapnic respiratory failur...


In [4]:
df_unique_texts = df_pmc_patients.drop_duplicates(subset='full_text')

In [5]:
df_unique_texts['full_text'].values

array(['Echocardiographic assessment and percutaneous closure of multiple atrial septal defects Atrial septal defect closure is now routinely performed using a percutaneous approach under echocardiographic guidance. Centrally located, secundum defects are ideal for device closure but there is considerable morphological variation in size and location of the defects. A small proportion of atrial septal defects may have multiple fenestrations and these are often considered unsuitable for device closure. We report three cases of multiple atrial septal defects successfully closed with two Amplatzer septal occluders. Introduction Atrial septal defect (ASD) closure is now commonly performed using a transcatheter, percutaneous approach and with the Amplatzer septal occluder, large defects can be safely closed. Device deployment requires a rim of atrial septal tissue surrounding the defect to allow effective capture of the septum by the occluder. The rim of tissue is also important to separate 

In [6]:
len(df_pmc_patients['full_text'].unique())

208357

In [7]:
len(df_pmc_patients['full_text'][200])

10179

# 1. Embedding model

In [37]:
from sentence_transformers import SentenceTransformer
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
import numpy as np

sentences = ["This is an example sentence", "Each sentence is converted"]

embeddings_model = HuggingFaceEmbeddings(model_name="neuml/pubmedbert-base-embeddings", 
                                        # model_kwargs={'device': self.device},
                                        encode_kwargs={"normalize_embeddings": True})
embeddings = np.array(embeddings_model.embed_documents(sentences))
print(embeddings)


[[-0.03628664 -0.00065551 -0.01743268 ... -0.0104356  -0.08590842
   0.05349242]
 [-0.07013984  0.0531532   0.03486726 ... -0.03975435 -0.07282241
   0.03394311]]


In [38]:
embeddings.min(), embeddings.max()

(-0.1594093143939972, 0.1335570365190506)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

embeddings_model = HuggingFaceEmbeddings(model_name="neuml/pubmedbert-base-embeddings", 
                                        # model_kwargs={'device': self.device},
                                        encode_kwargs={"normalize_L2": True})

text_splitter = SemanticChunker(embeddings_model)

semantic_chunks = text_splitter.create_documents([df_pmc_patients['full_text'][0]])

semantic_chunks[1]

Document(metadata={}, page_content='The patient remained well with no evidence of residual shunt six months following the procedure. Case 2 A 31-year old woman was found to have a secundum ASD during investigations for breathlessness. The defect was estimated to be 10 mm wide on TTE with evidence of right atrial and right ventricular dilatation. Left atrial size was normal.')

In [10]:
[chunk.page_content for chunk in semantic_chunks]

['Echocardiographic assessment and percutaneous closure of multiple atrial septal defects Atrial septal defect closure is now routinely performed using a percutaneous approach under echocardiographic guidance. Centrally located, secundum defects are ideal for device closure but there is considerable morphological variation in size and location of the defects. A small proportion of atrial septal defects may have multiple fenestrations and these are often considered unsuitable for device closure. We report three cases of multiple atrial septal defects successfully closed with two Amplatzer septal occluders. Introduction Atrial septal defect (ASD) closure is now commonly performed using a transcatheter, percutaneous approach and with the Amplatzer septal occluder, large defects can be safely closed. Device deployment requires a rim of atrial septal tissue surrounding the defect to allow effective capture of the septum by the occluder. The rim of tissue is also important to separate the se

In [11]:
for semantic_chunk in semantic_chunks:
    print(len(semantic_chunk.page_content), semantic_chunk)

3084 page_content='Echocardiographic assessment and percutaneous closure of multiple atrial septal defects Atrial septal defect closure is now routinely performed using a percutaneous approach under echocardiographic guidance. Centrally located, secundum defects are ideal for device closure but there is considerable morphological variation in size and location of the defects. A small proportion of atrial septal defects may have multiple fenestrations and these are often considered unsuitable for device closure. We report three cases of multiple atrial septal defects successfully closed with two Amplatzer septal occluders. Introduction Atrial septal defect (ASD) closure is now commonly performed using a transcatheter, percutaneous approach and with the Amplatzer septal occluder, large defects can be safely closed. Device deployment requires a rim of atrial septal tissue surrounding the defect to allow effective capture of the septum by the occluder. The rim of tissue is also important t

# 2. Indexing

https://medium.com/the-ai-forum/semantic-chunking-for-rag-f4733025d5f5

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1024,
    chunk_overlap=128,
    length_function=len,
    is_separator_regex=False,
)

In [6]:
texts = text_splitter.create_documents([df_pmc_patients['full_text'][0]])
print(len(texts))

for text in texts:
    print(len(text.page_content))

11
1016
1019
1021
1018
1023
1018
1018
1020
1021
1023
610


# 3. Query transformation

https://github.com/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/query_transformations.ipynb

## 3.1 rewrite query

In [3]:
from ollama import chat, ChatResponse

def rewrite_query(original_query: str):
    query_rewrite_template = f"""You are an AI assistant tasked with reformulating user queries to improve retrieval in a RAG system. 
    Given the original query, rewrite it to be more specific, detailed, and likely to retrieve relevant information. Do not add extra details, just return the rewritten query.

    Original query: {original_query}

    Rewritten query:"""

    response: ChatResponse = chat(model='llama3.2:3b', messages=[
        {
            'role': 'user',
            'content': query_rewrite_template,
        },
    ])

    return response.message.content


In [4]:
original_query = "What are the impacts of climate change on the environment?"
rewritten_query = rewrite_query(original_query)
print("Original query:", original_query)
print("\nRewritten query:", rewritten_query)

Original query: What are the impacts of climate change on the environment?

Rewritten query: What are the specific environmental consequences resulting from rising global temperatures, sea-level rise, and altered weather patterns caused by climate change?


## 3.2 Step-back Prompting: Generating broader queries for better context retrieval.


In [5]:
def generate_step_back_query(original_query):
    step_back_template = f"""You are an AI assistant tasked with generating broader, more general queries to improve context retrieval in a RAG system.
    Given the original query, generate a step-back query that is more general and can help retrieve relevant background information. Do not add extra details, just return the step-back query.

    Original query: {original_query}

    Step-back query:"""

    response: ChatResponse = chat(model='llama3.2:3b', messages=[
        {
            'role': 'user',
            'content': step_back_template,
        },
    ])

    return response.message.content


In [6]:
original_query = "What are the impacts of climate change on the environment?"
step_back_query = generate_step_back_query(original_query)
print("Original query:", original_query)
print("\nStep-back query:", step_back_query)

Original query: What are the impacts of climate change on the environment?

Step-back query: What are the environmental effects of global warming?


## 3.3 Sub-query Decomposition: Breaking complex queries into simpler sub-queries.

In [7]:
def decompose_query(original_query: str):
    subquery_decomposition_template = f"""You are an AI assistant tasked with breaking down complex queries into simpler sub-queries for a RAG system.
    Given the original query, decompose it into 2-4 simpler sub-queries that, when answered together, would provide a comprehensive response to the original query. Do not add extra details, just return the sub-queries.

    Original query: {original_query}

    example: What are the impacts of climate change on the environment?

    Sub-queries:
    1. What are the impacts of climate change on biodiversity?
    2. How does climate change affect the oceans?
    3. What are the effects of climate change on agriculture?
    4. What are the impacts of climate change on human health?"""

    response: ChatResponse = chat(model='llama3.2:3b', messages=[
        {
            'role': 'user',
            'content': subquery_decomposition_template,
        },
    ])

    return response.message.content


In [8]:
original_query = "How does ischemic stroke affect brain?"
sub_queries = decompose_query(original_query)
print("\nSub-queries:")
print(sub_queries)


Sub-queries:
Sub-queries:

1. What is ischemic stroke and its causes in the brain?
2. How do blood vessels function in the brain after an ischemic stroke?
3. What are the effects of a lack of oxygen in the brain due to an ischemic stroke?
4. Can ischemic strokes be treated or prevented with medical intervention?


# 4. Retrieval (Embedding model and Vector Database)

In [75]:
from langchain_community.vectorstores import FAISS
import faiss
import pandas as pd
from loguru import logger
import os
from langchain_core.documents import Document
import torch
from tqdm import tqdm
from typing import List, Tuple
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.docstore.in_memory import InMemoryDocstore


#https://www.anthropic.com/news/contextual-retrieval
#https://github.com/anthropics/anthropic-cookbook/tree/main/skills/contextual-embeddings

def convert_documents_to_text(results: List[Tuple[Document, float]]) -> str:
    context = "\n\n".join(doc.page_content for doc, score in results)
    return context

class VectorDB:
    def __init__(self, 
                 db_path: str = '/s3/misha/data_dir/PMC_patients/db_faiss'):
        self.db_path = db_path
        os.makedirs(self.db_path, exist_ok=True)

        self.index_path = os.path.join(self.db_path, "faiss_index.idx")
        self.metadata_path = os.path.join(self.db_path, "metadata.pkl")

        self.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        self.embeddings_model = HuggingFaceEmbeddings(model_name="neuml/pubmedbert-base-embeddings", 
                                                      model_kwargs={'device': self.device},
                                                      encode_kwargs={"normalize_embeddings": True})

    def create_knowledge_base(self, df: pd.DataFrame, batch_size: int = 1000):
        def document_generator():
            """Генератор документов, чтобы не хранить их в памяти"""
            for _, row in df.iterrows():
                yield Document(
                    page_content=row['chunk'],
                    metadata={'PMID': row['PMID'], 'title': row['title']}
                )

        logger.info('Initializing FAISS index...')
        embedding_dim = len(self.embeddings_model.embed_query("hello world"))
        index = faiss.IndexFlatIP(embedding_dim)  # Можно заменить на IndexIVFFlat для экономии памяти
        
        self.vector_store = FAISS(
            embedding_function=self.embeddings_model,
            index=index,
            docstore=InMemoryDocstore(),
            index_to_docstore_id={},
        )

        logger.info('Adding documents to FAISS...')
        buffer = []
        for i, doc in enumerate(tqdm(document_generator(), total=len(df))):
            buffer.append(doc)
            if len(buffer) >= batch_size:
                self.vector_store.add_documents(documents=buffer)
                buffer.clear()  # Очистка списка после загрузки в FAISS

        if buffer:  # Добавляем оставшиеся документы
            self.vector_store.add_documents(documents=buffer)

        logger.info('Saving vector store...')
        self.vector_store.save_local(self.db_path)
        logger.info('Vector store saved successfully!')

    def load_db(self):
        if not os.path.exists(self.db_path):
            raise ValueError("Vector database file not found. Use upload_data to create a new database.")

        self.vector_store = FAISS.load_local(self.db_path, 
                                             embeddings=self.embeddings_model, 
                                             allow_dangerous_deserialization=True)
        logger.info('vector database loaded from the local file successfully!')

    def search(self, query: str, k: int):
        if not self.vector_store:
            raise ValueError("Vector database is not created. Use create_knowledge_base() to create a new database.")
        results = self.vector_store.similarity_search_with_relevance_scores(query, k=k)
        results_rewritten = [{
            'PMID': result[0].metadata['PMID'],
            'title': result[0].metadata['title'],
            'chunk': result[0].page_content,
            # 'score': result[1]
        } for result in results]
        
        scores = [result[1] for result in results]  

        return results_rewritten, scores

In [76]:
vector_database = VectorDB()
# df_chunked_texts = pd.read_parquet('/s3/misha/data_dir/PMC_patients/chunked_texts.parquet', engine='pyarrow')
# vector_database.create_knowledge_base(df_chunked_texts)

In [77]:
vector_database.load_db()

2025-03-15 11:38:06.147 | INFO     | __main__:load_db:80 - vector database loaded from the local file successfully!


In [78]:
results_vector, scores_vector = vector_database.search(query='what is hemorrhagic stroke?', k=15)

In [79]:
results_vector

[{'PMID': '36514615',
  'title': 'Frontal Lobe Hemorrhage With Surrounding Edema and Subarachnoid Hemorrhage',
  'chunk': 'The etiologies, clinical presentation, and diagnosis of this often devastating type of stroke are presented. While she did have a significant neurologic deficit (neglect), she was able to remain alert and protect her airway. Her hospital course consisted of observation in the ICU and blood pressure management. The case illustrates that intracerebral hemorrhage (ICH) can sometimes present indolently and does not always require surgical intervention. Introduction Intracerebral hemorrhage (ICH) accounts for 15% of all strokes and 50% of stroke-related mortality, which equates to approximately 2.8 million deaths globally every year. In 2010, hemorrhagic strokes accounted for nearly a third of 33 million stroke cases and had a mortality rate of just over 50% worldwide. Two major risk factors for ICH are age and anticoagulant use. The overall increase in life expectancy 

In [80]:
scores_vector

[0.5783277770253581,
 0.5896253662092488,
 0.6158262917608341,
 0.6276033434933848,
 0.6339680655275604,
 0.638483636730169,
 0.6402704945196369,
 0.6445027124597068,
 0.6460534214569682,
 0.6504988603036547,
 0.6513528186747581,
 0.6521075844378912,
 0.6561155389970348,
 0.657551987887983,
 0.6589510103774536]

In [81]:
texts_vector = [result['chunk'] for result in results_vector]

# 5. Keyword search

In [69]:
import os
import json
import bm25s
import Stemmer
import pandas as pd
import logging

logger = logging.getLogger(__name__)

class KeywordDB:
    def __init__(self, db_path: str = '/s3/misha/data_dir/PMC_patients/db_bm25s'):
        self.db_path = db_path
        self.stemmer = Stemmer.Stemmer("english")
        self.retriever = None
        # self.index_to_metadata = {}
    
    def create_knowledge_base(self, df: pd.DataFrame):
        logger.info('Sampling 100k chunks...')
        df = df.sample(n=100000, random_state=42)
        
        logger.info('Tokenizing chunks...')
        corpus = df[['chunk', 'PMID', 'title']].to_dict(orient='records')
        corpus_texts = [doc['chunk'] for doc in corpus]

        corpus_tokens = bm25s.tokenize(corpus_texts, stopwords="en", stemmer=self.stemmer)
        self.retriever = bm25s.BM25()
        self.retriever.index(corpus_tokens)
        
        os.makedirs(self.db_path, exist_ok=True)
        self.retriever.save(self.db_path, corpus=corpus_texts)
        
        with open(f'{self.db_path}/corpus.jsonl', 'w') as f:
            for record in corpus:
                f.write(json.dumps(record) + '\n')
        
        logger.info('Knowledge base created successfully!')
    
    def load_db(self):
        if not os.path.exists(self.db_path):
            raise ValueError("Keyword database file not found. Use create_knowledge_base to create a new database.")
        
        self.retriever = bm25s.BM25.load(self.db_path, load_corpus=True)
        with open(f'{self.db_path}/corpus.jsonl', 'r') as f:
            self.bm25s_corpus = [json.loads(line) for line in f]
        
        # self.index_to_metadata = {i: doc for i, doc in enumerate(self.bm25s_corpus)}
        logger.info('keyword database loaded successfully!')
    
    def search(self, query: str, k: int):
        if self.retriever is None:
            raise ValueError("Keyword database is not loaded. Use load_db() to load the database.")
        
        query_tokens = bm25s.tokenize(query, stemmer=self.stemmer)
        results, scores = self.retriever.retrieve(query_tokens, k=k)
        results = results[0].tolist()
        scores = scores[0].tolist()
        
        return results, scores

In [70]:
keyword_database = KeywordDB()

In [ ]:
# keyword_database.create_knowledge_base(df_pmc_patients)

Splitting texts using SemanticChunker:   0%|          | 0/100 [00:00<?, ?it/s]

Splitting texts using SemanticChunker: 100%|██████████| 100/100 [01:24<00:00,  1.19it/s]
Finding newlines for mmindex: 100%|██████████| 1.59M/1.59M [00:00<00:00, 149MB/s]


In [71]:
keyword_database.load_db()

In [72]:
query='what is hemorrhagic stroke?'

results_keyword, scores_keyword = keyword_database.search(query, k=15)

In [73]:
results_keyword

[{'chunk': "Patient Consent Written informed consent for all pertinent details of this case was obtained from patient's representative. References The role of dual energy CT in differentiating between brain haemorrhage and contrast medium after mechanical revascularisation in acute ischaemic stroke Dual-energy CT: what the neuroradiologist should know Dual- and multi-energy CT: principles, technical approaches, and clinical applications Energy-selective reconstructions in x-ray computerised tomography Material-selective imaging and density measurement using the dual-energy method. I. Principles and methodology A quantitative theory of the Hounsfield unit and its application to dual energy scanning First performance evaluation of a dual-source CT (DSCT) system Material separation using dual-energy CT: current and emerging applications Dual-energy CT to differentiate small foci of intracranial hemorrhage from calcium Dual-energy CT follow-up after stroke thrombolysis alters assessment of

In [74]:
scores_keyword

[6.751938819885254,
 6.06105899810791,
 6.055565357208252,
 6.0503926277160645,
 5.937287330627441,
 5.870580196380615,
 5.796564102172852,
 5.733941078186035,
 5.693838119506836,
 5.610304832458496,
 5.594485282897949,
 5.5315704345703125,
 5.434269905090332,
 5.414670467376709,
 5.390836238861084]

In [84]:
texts_keyword = [result['chunk'] for result in results_keyword]

# 6. Reranking

In [86]:
results_vector

[{'PMID': '36514615',
  'title': 'Frontal Lobe Hemorrhage With Surrounding Edema and Subarachnoid Hemorrhage',
  'chunk': 'The etiologies, clinical presentation, and diagnosis of this often devastating type of stroke are presented. While she did have a significant neurologic deficit (neglect), she was able to remain alert and protect her airway. Her hospital course consisted of observation in the ICU and blood pressure management. The case illustrates that intracerebral hemorrhage (ICH) can sometimes present indolently and does not always require surgical intervention. Introduction Intracerebral hemorrhage (ICH) accounts for 15% of all strokes and 50% of stroke-related mortality, which equates to approximately 2.8 million deaths globally every year. In 2010, hemorrhagic strokes accounted for nearly a third of 33 million stroke cases and had a mortality rate of just over 50% worldwide. Two major risk factors for ICH are age and anticoagulant use. The overall increase in life expectancy 

In [87]:
results_keyword

[{'chunk': "Patient Consent Written informed consent for all pertinent details of this case was obtained from patient's representative. References The role of dual energy CT in differentiating between brain haemorrhage and contrast medium after mechanical revascularisation in acute ischaemic stroke Dual-energy CT: what the neuroradiologist should know Dual- and multi-energy CT: principles, technical approaches, and clinical applications Energy-selective reconstructions in x-ray computerised tomography Material-selective imaging and density measurement using the dual-energy method. I. Principles and methodology A quantitative theory of the Hounsfield unit and its application to dual energy scanning First performance evaluation of a dual-source CT (DSCT) system Material separation using dual-energy CT: current and emerging applications Dual-energy CT to differentiate small foci of intracranial hemorrhage from calcium Dual-energy CT follow-up after stroke thrombolysis alters assessment of

In [89]:
results_combined = results_vector + results_keyword

In [101]:
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np
from typing import List

device = torch.device('cuda:0')
model = HuggingFaceEmbeddings(model_name="neuml/pubmedbert-base-embeddings", 
                                        model_kwargs={'device': device},
                                        encode_kwargs={"normalize_embeddings": True})
def rerank_documents(query: str, retrieved_results: List[str], model: HuggingFaceEmbeddings, top_k: int = 15):
    retrieved_docs = [result['chunk'] for result in retrieved_results]
    query_embedding = model.embed_query(query)
    doc_embeddings = model.embed_documents(retrieved_docs)

    similarity_scores = np.dot(doc_embeddings, query_embedding)  # Vector dot product
    reranked_docs = sorted(zip(retrieved_docs, similarity_scores), key=lambda x: x[1], reverse=True)

    top_k_reranked_docs = reranked_docs[:top_k]
    top_k_reranked_texts = [doc[0] for doc in top_k_reranked_docs]

    top_k_reranked_results = [result for result in retrieved_results if result['chunk'] in top_k_reranked_texts]
    top_k_reranked_scores = [doc[1] for doc in top_k_reranked_docs]

    return top_k_reranked_results, top_k_reranked_scores

In [102]:
reranked_results, reranked_scores = rerank_documents(query='what is hemorrhagic stroke?', retrieved_results=results_combined, model=model)

In [103]:
reranked_results

[{'PMID': '36514615',
  'title': 'Frontal Lobe Hemorrhage With Surrounding Edema and Subarachnoid Hemorrhage',
  'chunk': 'The etiologies, clinical presentation, and diagnosis of this often devastating type of stroke are presented. While she did have a significant neurologic deficit (neglect), she was able to remain alert and protect her airway. Her hospital course consisted of observation in the ICU and blood pressure management. The case illustrates that intracerebral hemorrhage (ICH) can sometimes present indolently and does not always require surgical intervention. Introduction Intracerebral hemorrhage (ICH) accounts for 15% of all strokes and 50% of stroke-related mortality, which equates to approximately 2.8 million deaths globally every year. In 2010, hemorrhagic strokes accounted for nearly a third of 33 million stroke cases and had a mortality rate of just over 50% worldwide. Two major risk factors for ICH are age and anticoagulant use. The overall increase in life expectancy 

In [104]:
reranked_scores

[0.5963346058781136,
 0.5803573709399737,
 0.5803573709399737,
 0.5433037916997874,
 0.5266483565352923,
 0.51764731154658,
 0.51764731154658,
 0.5112614875074295,
 0.5087343267532095,
 0.502749074523404,
 0.5005559643622766,
 0.49426924096184893,
 0.49306145832813425,
 0.49199416039241123,
 0.48632618215787343]

# 7. Summarization

In [115]:
from sentence_transformers import SentenceTransformer, util
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.parsers.plaintext import PlaintextParser


class QueryBasedTextRankSummarizer:
    def __init__(self):
        self.embedding_model = SentenceTransformer('neuml/pubmedbert-base-embeddings')
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.embedding_model.to(self.device)

    def process(self, query: str, reranked_records: List[dict]):
        context_snippets = [record['chunk'] for record in reranked_records if record['chunk'].strip()]
        pmids = [record['PMID'] for record in reranked_records]
        titles = [record['title'] for record in reranked_records]
        if not context_snippets:
            raise ValueError("context_snippets must be a non-empty list of valid sentences.")

        query_embedding = self.embedding_model.encode(query, convert_to_tensor=True).to(self.device)
        sentence_embeddings = self.embedding_model.encode(context_snippets, convert_to_tensor=True).to(self.device)

        similarities = util.cos_sim(query_embedding, sentence_embeddings).squeeze().tolist()

        ranked_sentences = sorted(
            zip(similarities, context_snippets), reverse=True, key=lambda x: x[0]
        )

        count_ranked_sentences = len(ranked_sentences)
        filtered_context = " ".join([sentence for _, sentence in ranked_sentences[:int(count_ranked_sentences / 2)]])

        summary_sentence_count = max(1, round(0.2 * count_ranked_sentences))
        parser = PlaintextParser.from_string(filtered_context, Tokenizer("english"))
        summarizer = TextRankSummarizer()
        summary = summarizer(parser.document, summary_sentence_count)

        summarized_text = ''.join(sentence._text for sentence in summary)
        return summarized_text, pmids, titles


In [116]:
query_based_summarizer = QueryBasedTextRankSummarizer()

In [117]:
summarized_texts, pmids, titles = query_based_summarizer.process(query='what is hemorrhagic stroke?', reranked_records=reranked_results)
len(summarized_texts)

3231

In [118]:
pmids

['36514615',
 '33880292',
 '32702937',
 '36721551',
 '21831285',
 '28503483',
 '26005573',
 '29682058',
 '31215417',
 '34513463',
 '27847773',
 '33390937',
 '31723423',
 '21831285',
 '33880292']

In [119]:
titles

['Frontal Lobe Hemorrhage With Surrounding Edema and Subarachnoid Hemorrhage',
 'A Case Report Examining a Contraindication for Mechanical Thrombectomy in the Setting of a Large Vessel Occlusion and a Concurrent Contralateral Intracranial Hemorrhage',
 'Computed tomography-negative symptomatic intracerebral hemorrhage in a patient with cerebral small vessel disease',
 'Implementation of the Gamut of Physiotherapy Maneuvers in Restoration and Normalization of Functional Potencies in a Patient With a Hemorrhagic Stroke: A Case Report',
 'Recurrent, sequential, bilateral deep cerebellar hemorrhages: a case report',
 'Importance of Hematoma Removal Ratio in Ruptured Middle Cerebral Artery Aneurysm Surgery with Intrasylvian Hematoma',
 'The Dumbest Mistake I Ever Made',
 'Hemispheric Infarct Following a Cerebellar Hematoma: A Rare Coincidence',
 'Acute ischemic stroke with contralateral convexal subarachnoid hemorrhage: two cases report',
 'Acute Headache Due to Intracerebral Hemorrhage Sec

# 8. QA Pipeline

In [141]:
from ollama import chat, ChatResponse
from langchain_huggingface import HuggingFaceEmbeddings


class QAPipeline:
    def __init__(self, 
                 vector_database: VectorDB,
                 keyword_database: KeywordDB,
                 summarizer: QueryBasedTextRankSummarizer,
                 embedding_model: HuggingFaceEmbeddings,
                 k: int = 15):
        self.vector_database = vector_database
        self.keyword_database = keyword_database
        self.summarizer = summarizer
        self.embedding_model = embedding_model
        self.k = k


    def generate_response(self, query: str):
        #vector search
        results_vector, scores_vector  = self.vector_database.search(query, k=self.k)

        #keyword search
        results_keyword, scores_keyword = self.keyword_database.search(query, k=self.k)

        #reranking
        results_combined = results_vector + results_keyword
        reranked_results, reranked_scores = rerank_documents(query, retrieved_results=results_combined, model=self.embedding_model)

        # summarize texts
        summarized_texts, pmids, titles = self.summarizer.process(query, reranked_records=reranked_results)

        prompt = f"""
            Question: {query}\n\n
            Context to use to provide health recommendations divided into paragraphs and generated lists:\n\n
            {summarized_texts}\n\n
            Sources:\n
            {chr(10).join([f"- {title} (PMID: {pmid})" for title, pmid in zip(titles, pmids)])}\n\n
            If the context does not contain the answer to the question, write 'The suggested context does not contain the answer to the question', and try to answer on your own, giving references to the sources you used. But do not make up anything—use only factual and trustworthy data.
        """

        response: ChatResponse = chat(model='llama3.2:3b', messages=[
            {
                "role": "system", 
                "content": """
                    You are an expert in producing health recommendations based on given content.    
                """
            },
            {
                "role": "user",
                "content": prompt
            },
        ])

        return response.message.content

In [142]:
qa_pipeline = QAPipeline(vector_database, keyword_database, query_based_summarizer, embedding_model=model)

In [143]:
with open('/s3/misha/data_dir/PMC_patients/PMC-Patients-ReCDS/queries/train_queries.jsonl', 'r') as f:
    train_queries = [json.loads(line) for line in f]

In [144]:
train_queries[1000]['text']

"A 47-year-old patient with an enormous uterine leiomyoma reaching beyond the navel and up to the costal arch was admitted. During the 14 years since its detection, because of the patient's extreme fear of an abdominal incision, the myoma was merely monitored and all suggested laparotomies thus far had been refused. At the moment of admission, the patient only agreed to a minimally invasive surgery. She was informed in detail about all risks, side effects, and alternatives as well as the potential risk for an emergency open abdominal surgery. Before surgery, we performed imaging diagnostics by means of computed tomography (CT) of the abdomen ().\nWhen performing a hysterectomy of a very large uterus (>2500 g), the anatomical changes in the abdomen caused by the size of the uterus need to be taken into account. The large uterus divides the abdominal area, and only 3 narrow spaces are left to manipulate surgical instruments: between the left uterine wall and left abdominal wall, between 

In [149]:
query = 'what is aboba?'
response = qa_pipeline.generate_response(query)

In [150]:
print(query)

what is aboba?


In [151]:
print(response)

I have searched through the provided context and sources. Unfortunately, I was unable to find any information that directly answers the question "what is Aboba?" in the given sources.

The suggested context does not contain the answer to the question.

However, after conducting an external search (using only factual and trustworthy data), I found that "Aboba" refers to a popular Nigerian dish made from yam flour. If you are interested in learning more about this dish or its health benefits, I can provide general information on the nutritional content of Aboba based on available online sources.

Please let me know if you would like me to proceed with providing general information on Aboba's nutrition and potential health implications.
